In [ ]:
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import GPy
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from ipywidgets import interact

In [ ]:
'''load datasets 1: toy'''
N = 100  # train + test
noise_std = 0.05
true_f = lambda xx: xx*np.sin(xx*10.)  # true underlying function

toy_vis_x = np.linspace(0,1,100).reshape((-1,1))
toy_vis_y = true_f(toy_vis_x)
X = np.random.rand(N,1)
Y = true_f(X) + noise_std * np.random.randn(X.shape[0]).reshape((-1,1))  # with additive noise
toy_train_x, toy_test_x, toy_train_y, toy_test_y = train_test_split(X, Y, test_size=0.8)


'''laod datasets 2: Boston'''
boston = load_boston()
bo_train_x, bo_test_x, bo_train_y, bo_test_y = train_test_split(boston.data, boston.target, test_size=0.4)

In [ ]:
print toy_train-

# Toy Example : GP vs GB

In [ ]:
toy_train_x.shape
toy_train_y.shape

In [ ]:
plt.plot(toy_vis_x, toy_vis_y)
plt.plot(toy_train_x, toy_train_y, 'k+', markersize=11)

In [ ]:
kernel = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)
m = GPy.models.GPRegression(toy_train_x, toy_train_y, kernel)
m.optimize(messages=True)
print m.kern.lengthscale
print m.kern.variance
print m.Gaussian_noise.gaussian_variance
mu_f, var_f  = m.predict(toy_vis_x)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
gbr = GridSearchCV(GradientBoostingRegressor(), {'n_estimators':[10,50,100,200]}, cv=5)
gbr.fit(toy_train_x, toy_train_y)
gbr_f = gbr.predict(toy_vis_x)

In [ ]:
plt.plot(toy_vis_x, toy_vis_y)
plt.plot(toy_vis_x, mu_f, label='Gaussian Process')
plt.plot(toy_vis_x, gbr_f, label='Gradient Boosting')
plt.plot(toy_train_x, toy_train_y, 'k+', markersize=11)
plt.legend(loc='best')

# The Effect of Hyperparameters

In [ ]:
@interact(leng=(0.1,1.,0.05), var=(0.1,1,0.05), noise=(0.001,0.1,0.005))
def vis(leng, var, noise):
    kernel = GPy.kern.RBF(input_dim=1, variance=var, lengthscale=leng)
    m = GPy.models.GPRegression(toy_train_x, toy_train_y, kernel)
    m.Gaussian_noise.fix(noise)
    
    print m.kern.lengthscale
    print m.kern.variance
    print m.Gaussian_noise.gaussian_variance
    mu_f, var_f  = m.predict(toy_vis_x)
    plt.plot(toy_vis_x, toy_vis_y, label='true')
    plt.plot(toy_train_x, toy_train_y, 'k+', markersize=11)
    plt.plot(toy_vis_x, mu_f, label='GPR')
    plt.legend(loc='best')